In [18]:
import subprocess
import boto3
import os
from google.cloud import storage
import shutil

In [19]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name,timeout=None)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name,timeout=None)

def processRTC(s3key,gcskey):
    if os.path.exists('./temp/'):
        shutil.rmtree('./temp/')
    os.mkdir('./temp/')
    os.mkdir('./temp/deflate/')

    #Download RTC file
    print('Downloading RTC File')
    session = boto3.session.Session(profile_name='saml-pub')
    bucket = 'opera-int-rs-fwd'
    filename = s3key.split('/')[-1]
    filepath = './temp/'+filename
    s3 = session.client('s3')
    s3.download_file(bucket, s3key, filepath)

    #Change compression to DEFLATE
    print('Translating compression')
    outfile = './temp/deflate/'+filename
    gdal_cmd = f'gdal_translate -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 {filepath} {outfile}'
    subprocess.run(gdal_cmd,shell=True,)

    #Upload to GCS bucket
    print('Uploading to GCS Bucket')
    gcsbucket = "opera-bucket-rtc"
    upload_blob(gcsbucket,outfile,gcskey)


In [20]:
processRTC('products/RTC_S1/OPERA_L2_RTC-S1_T121-258596-IW1_20220411T000836Z_20230503T114347Z_S1A_30_v0.0/OPERA_L2_RTC-S1_T121-258596-IW1_20220411T000836Z_20230503T114347Z_S1A_30_v0.0_VH.tif','products/test-run/OPERA_L2_RTC-S1_T121-258596-IW1_20220411T000836Z_20230503T114347Z_S1A_30_v0.0_VH.tif')

Translating compression
Input file size is 3307, 1513
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket


In [21]:
session = boto3.session.Session(profile_name='saml-pub')
client = session.client('s3')
result = client.list_objects(Bucket='opera-int-rs-fwd', Prefix='products/RTC_S1/', Delimiter='/')
keyList = []
for o in result.get('CommonPrefixes'):
    path = o.get('Prefix')
    if path[-2] != 's':  #checks if end is not "static layers"
        fname = path.split('/')[-2]+'_VH.tif'
        keyList.append(path+fname)

    # print('sub folder : ', o.get('Prefix'))

In [22]:
for key in keyList:
    fname = key.split('/')[-1]
    gcsKey = 'products/test-run/'+fname
    processRTC(key,gcsKey)

Translating compression
Input file size is 3307, 1513
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Translating compression
Input file size is 3435, 1492
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Translating compression
Input file size is 3091, 1481
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Translating compression
Input file size is 3306, 1514
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Translating compression
Input file size is 3435, 1494
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Translating compression
Input file size is 3091, 1483
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Translating compression
Input file size is 3306, 1516
0...10...20...30...40...50...60...70...80...90...100 - done.
Uploading to GCS Bucket
Translating compression
Inp

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found